<a href="https://colab.research.google.com/github/budzinskaya/LiveCorpus/blob/master/quickpass_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Программистская часть

#### Задача 1. 

Напишите черновик игры в жанре RPG. Идеологически: игрок выбирает, будет ли он играть за волшебника или за бойца, а потом выбранным героем сражается с монстрами, набирая очки опыта. Что должно быть технически:

- классы волшебника и бойца (можно создать отдельный класс Player и наследоваться от него, но необязательно)
- класс монстра (хотя бы один)
- класс оружия (тут тоже фантазию не ограничиваю - можно создать абстрактный класс и наследоваться от него, можно сделать классы для меча и для посоха с варьирующими атрибутами)
- класс Игра, в котором будут все необходимые методы
- все это должно быть разложено по отдельным скриптам .py в папке, класс игры импортируется в файл main.py, и его методы вызываются там. 

In [ ]:
# your code here

#### Задача 2. 

Дан текст, каждая строка которого является полным или относительным путём к некоторому файлу.
Напишите регулярное выражение, которое захватывает:
1. директорию, в которой лежит файл;
2. только имя файла (без расширения);
3. только расширение;
При этом:
- нужны только файлы, у которых расширение не .bat и не .txt.
- пути могут быть как в Unix, так и в Windows формате (https://ru.wikipedia.org/wiki/Путь_к_файлу).
- расширение, если оно есть, начинается с точки. Файлы могут быть без расширения вовсе (в этом случае на месте расширения должно стоять None или "")
- скрытые файлы могут начинаться с точки (например, .bashrc - и это не расширение)
- относительный путь может содержать только название файла, в этом случае вместо директории выведите None или ""
- в остальных случаях директория должна заканчиаться на разделитель директорий. Наприемр, в Unix-системах - "/" - это путь к корневой директории.
Требуется получить список из кортежей, каждый кортеж содержит извлечённые данные.
Используйте флаг VERBOSE, чтобы не запутаться.
(Расширение в целом может содержать всё, что угодно, но разделителей директорий не может быть в именах файлах и расширениях. https://en.wikipedia.org/wiki/List_of_filename_extensions )

In [9]:
import re

def process_paths(paths):
    pattern = re.compile(r'(?m)' +
                         r'^([\\/]?(?:[^\\/:*?\"<>|\r\n]+[\\/])*)' +
                         r'(\.?(?:[^\\/:*?\"<>|\r\n\.]|\.(?=.*\.))+)' +
                         r'(?!\.bat(?:$|\n))(?!\.txt(?:$|\n))' +
                         r'(\.[^\\/:*?\"<>|\r\n\.]+)?$')
    return re.findall(pattern, paths)

paths = '''
file.txt
file.txts
\\dir\\file.bat
\\dir\\file.bats
/dir/dir/file.py
dir\\file.py
dir\\file.before.py
.file
/dir/.file
file.py
/file.py
/file.py/file.py
dir1/file 1.7z
'''

expected_results = [
    ('', 'file', '.txts'),
    ('\\dir\\', 'file', '.bats'),
    ('/dir/dir/', 'file', '.py'),
    ('dir\\', 'file', '.py'),
    ('dir\\', 'file.before', '.py'),
    ('', '.file', ''),
    ('/dir/', '.file', ''),
    ('', 'file', '.py'),
    ('/', 'file', '.py'),
    ('/file.py/', 'file', '.py'),
    ('dir1/', 'file 1', '.7z'),
]

actual_results = process_paths(paths)

assert(actual_results == expected_results)

#### Задача 3. 

Жизнь.
Напишите игру "Жизнь".
Что это такое - читайте в википедии и здесь: http://www.michurin.net/online-tools/life-game.html
Вообще говоря, это не игра в привычном понимании этого слова, а процесс.
В простейшем виде достаточно раз в 0.1 секунды выводить на экран обновлённое поле. Для рамочек можно использовать специальные символы для рисования рамочек (найдите в таблице unicode). Пробел - пустая клетка, живая клетка может быть обозначена, например, символом '+'. Начальное поле генерируется случайным образом, вероятность появления жизни в клетке при начальной генерации - должна быть настраиваемым параметром. Размеры поля вводит пользователь при запуске программы. Также должна быть возможность в качестве начальной популяции использовать R-pentomino (http://www.conwaylife.com/wiki/R-pentomino)

In [1]:
import numpy as np
import time

from IPython.display import clear_output

class Life:
    def __init__(self, width=60, height=40, prob=0.3, time=0.1):
        self._prob = prob
        self._width = width
        self._height = height
        self._time = time

    def _initialize_field(self, randomly=True):
        if randomly:
            self._field = np.random.random((self._height, self._width)) < self._prob
        else:
            self._field = np.full((self._height, self._width), False)
            cx = self._width // 2
            cy = self._height // 2
            self._field[cy - 1, cx] = True
            self._field[cy - 1, cx + 1] = True
            self._field[cy, cx] = True
            self._field[cy, cx - 1] = True
            self._field[cy + 1, cx] = True
        
        # массив для подсчёта живых соседей с "рамкой"
        self._scores = np.zeros((self._height + 2, self._width + 2))

    def _one_step(self):
        self._scores.fill(0)
        # сместим field 8 раза в разных направлениях и прибавим к scores
        for i in range(3):
            for j in range(3):
                if i == 1 and j == 1:
                    continue

                self._scores[i: i + self._height, j: j + self._width] += self._field

        # мёртвые клетки с 3 живыми соседями
        condition1 = (~self._field) & (self._scores[1: self._height + 1, 1: self._width + 1] == 3)
        # живые клетки с 2 или 3 живими соседями
        condition2 = self._field & ((self._scores[1: self._height + 1, 1: self._width + 1] == 2) | (self._scores[1: self._height + 1, 1: self._width + 1] == 3))

        self._field = condition1 | condition2
        

    def _show_field(self):
        horizontal_line = u'\u2501'
        vertical_line = u'\u2503'
        top_left_corner = u'\u250F'
        top_right_corner = u'\u2513'
        bottom_left_corner = u'\u2517'
        bottom_right_corner = u'\u251B'

        result = top_left_corner + horizontal_line * self._width + top_right_corner + '\n'
        for i in range(self._height):
            result += vertical_line + ''.join('+' if x else ' ' for x in self._field[i]) + vertical_line + '\n'
        result += bottom_left_corner + horizontal_line * self._width + bottom_right_corner
        clear_output(wait=True)
        print(result)


    def run(self, randomly=True):
        self._initialize_field(randomly)
        while True: 
            self._show_field()
            time.sleep(self._time)
            self._one_step()


In [4]:
print("Введите через пробел ширину и высоту поля")
size = input().split()
width, height = int(size[0]), int(size[1])
print("Введите R, если хотите использовать в качестве начальной популяции R-pentomino (иначе любое другое значение)")
randomly = input() != "R"
if randomly:
    print("Введите вероятность появления жизни")
    prob = float(input())
    life = Life(width, height, prob)
else:
    life = Life(width, height)
life.run(randomly)


┏━━━━━━━━━━┓
┃          ┃
┃          ┃
┃          ┃
┃          ┃
┃          ┃
┃          ┃
┃    +     ┃
┃   + +    ┃
┃    +     ┃
┃          ┃
┗━━━━━━━━━━┛


KeyboardInterrupt: ignored

### Лингвистическая часть

Для выполнения этих заданий выберите два любых достаточно длинных текста (.txt) на русском и на любом другом (для которого есть парсеры) языке; если возьмете текст и его перевод, будет отлично.

#### Задача 4. 

Просмотрите оба выбранных текста. Удостоверьтесь, что тексты чистые, если же в них есть какой-то мусор: хештеги, затесавшиеся при OCR символы и подобное, почистите с помощью регулярных выражений. 

Проведите первичный статистический анализ: разбейте тексты на предложения и на токены, посчитайте относительное количество того и другого, сопоставьте. Если ваши тексты параллельные, какой длиннее? В каком тексте средняя длина предложения больше? Почему? В каком тексте выше лексическое разнообразие? 

Таким образом, вам необходимо узнать следующие вещи:

- количество предложений (относительное и абсолютное)
- количество токенов (относительное и абсолютное)
- средняя длина предложения (среднее количество слов в предложении)
- соотношение "уникальные токены / все токены"
- (опционально) соотношение знаков пунктуации и слов

In [5]:
!pip install nltk
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
def preprocess_text(filepath, regex):
    with open(filepath, 'r') as file:
        text = file.read()
        text = re.sub(regex, '', text)
        text = re.sub(r'\s+', ' ', text)
        
        return text


In [47]:
from tabulate import tabulate
from nltk.tokenize import sent_tokenize, word_tokenize

def analyze_text(text, word_regex):
    sents = sent_tokenize(text)
    tokens = word_tokenize(text)

    reg = re.compile(word_regex)
    words = [token for token in tokens if re.fullmatch(reg, token)]

    sent_lengths = [len(word_tokenize(sent)) for sent in sents]
    mean_length = sum(sent_lengths) / len(sent_lengths)

    results = {
        'кол-во предложений': len(sents),
        'кол-во токенов': len(tokens),
        'средняя длина предложения': mean_length,
        'уникальные токены/все токены': len(set(tokens)) / len(tokens),
        'знаки пунктуации/слова': (len(tokens) - len(words)) / len(words)
    }

    return results


text_ru = preprocess_text('prestuplenie-i-nakazanie.txt', r'[^А-Яа-яЁё.,:;!?–—\-()"\s]+')
text_de = preprocess_text('verbrechen_und_strafe.txt', r'[^A-Za-zÄäÖöÜüß.,:;!?–—\-()"\s]+')

analysis_ru = analyze_text(text_ru, r'[А-Яа-яЁё\-]+')
analysis_de = analyze_text(text_de, r'[A-Za-zÄäÖöÜüß]+')

table_data = [
    ['', 'ru', 'de', 'ru/de'],
]

for key in analysis_ru.keys():
    table_data.append([key, analysis_ru[key], analysis_de[key], analysis_ru[key] / analysis_de[key]])

print(tabulate(table_data, headers="firstrow", tablefmt="grid"))

+------------------------------+---------------+----------------+----------+
|                              |            ru |             de |    ru/de |
+==============================+===============+================+==========+
| кол-во предложений           |  13692        |  14307         | 0.957014 |
+------------------------------+---------------+----------------+----------+
| кол-во токенов               | 218899        | 246203         | 0.8891   |
+------------------------------+---------------+----------------+----------+
| средняя длина предложения    |     15.9888   |     17.2086    | 0.929116 |
+------------------------------+---------------+----------------+----------+
| уникальные токены/все токены |      0.12662  |      0.0682079 | 1.85638  |
+------------------------------+---------------+----------------+----------+
| знаки пунктуации/слова       |      0.284717 |      0.193405  | 1.47213  |
+------------------------------+---------------+----------------+----------+

#### Задача 5. 

Сделайте морфосинтаксические разборы ваших текстов в формате UD, запишите .conllu-файлы. 

In [17]:
!pip install pymorphy2

!pip install -U pymorphy2-dicts-ru

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 71.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=f4210b2cabbb262693b933027de48ff96a80fd56553ffd772ac55b86023f0333
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
!pip install conllu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
import conllu

with open("mynew.conllu", "w") as f:
    for i in parse:
        f.write(i)
        f.write('\n\n')

TypeError: ignored

In [24]:
with open("prestuplenie.conllu", "w") as f:
        f.write(parse)
        f.write('\n\n')

TypeError: ignored

#### Задача 6. 

Посчитайте статистику по частям речи, сопоставьте: можно напечатать две таблички с процентами по частям речи. 

In [ ]:
# your code here

#### Задача 7. 

Посчитайте, какое соотношение токенов по частям речи имеет совпадающие со словоформой леммы (т.е., в скольких случаях токены с частью речи VERB, например, имели словарную форму: и сам токен, и лемма одинаковые). Что вы можете сказать о выбранных вами языках на основании этих данных? Ожидаются две таблички с процентами несовпадающих по лемме и токену слов для каждой части речи. 

In [ ]:
# your code here

#### Задача 8. 

Посчитайте медианную длину предложения для ваших текстов (медиана - это если взять все длины всех ваших предложений, упорядочить их от маленького к большому и выбрать то число, которое оказалось посередине, а если чисел четное количество, то взять среднее арифметическое двух чисел посередине. Например, если у вас пять предложений длинами 1, 2, 6, 7, 8, то медиана - 6, а если шесть предложений длинами 1, 1, 7, 9, 10, 11, то медиана - (7 + 9) / 2 = 8). Возьмите любые два предложения (одно русское и второе на другом языке) и постройте для них деревья зависимостей. Изучите связи зависимостей (deprel) и вершины: согласны ли вы с разбором?

In [ ]:
# your code here

#### Задача 9. 

Посчитайте частотные списки токенов для каждой категории связей зависимостей (т.е., нужно выделить все токены в тексте, которые получали, например, ярлык amod, и посчитать их частоты). Выведите по первые три самых частотных токена для каждой категории (punct можно не выводить). 

In [ ]:
# your code here

#### Задача 10. 

Некоторые предлоги в русском языке могут управлять разными падежами (например, "я еду в Лондон" vs "я живу в Лондоне"). Давайте проанализируем эти предлоги и их падежи. Необходимо:

- составить список таких предлогов (РГ-80 вам в помощь)
- взять достаточно большой текст (можно большое художественное произведение)
- сделать морфоразбор этого текста (лучше не pymorphy)
- Посчитать, как часто и какие падежи встречаются у слова, идущего после предлога.

Примечания: во-первых, имейте в виду, что иногда после предлога могут идти самые неожиданные вещи: "я что, должен ехать на, черт побери, северный полюс?". Во-вторых, неплохо бы учитывать отсутствие пунктуации (конечно, в норме, как нам кажется, предлог обязательно требует зависимое, но! "да иди ты на!") Эти штуки можно отсеять, если просто учитывать только заранее определенные падежи, а не считать все, какие встретились (так и None можно огрести).

Если будете использовать RNNMorph, возможно, понадобится регулярное выражение и немного терпения.

In [ ]:
# your code here